# Генетические алгоритмы

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import math
import time

## Немного теории

**Генетические алгоритмы** (ГА) основаны на **эволюционном подходе** к ИИ, в котором методы эволюции используются для получения оптимального решения некоторой задачи. Они были предложены в 1975 году [Джоном Генри Холландом](https://en.wikipedia.org/wiki/John_Henry_Holland).

Генетические алгоритмы основаны на следующих идеях:
* Допустимые решения проблемы могут быть представлены в виде **генов**
* **Скрещивание** (или **кроссинговер**) позволяет нам объединить два решения вместе, чтобы получить новое работающее решение.
* **Отбор** используется для выбора более оптимальных решений с использованием некоторой функции приспособленности.
* **Мутации** вводятся, чтобы дестабилизировать оптимизацию и вывести нас из локального минимума

Для применения генетического алгоритма к решению задачи, необходимо:

 * Найти метод кодирования решения нашей задачи с использованием **генов** $g\in\Gamma$ - обычно геном кодируется некоторым вектором
 * На множестве генов $\Gamma$ нужно определить **функцию приспособленности** $\mathrm{fit}: \Gamma\to\mathbb{R}$. Наименьшие
значения функции будут соответствовать лучшим решениям.
 * Определить механизм **скрещивания** (или **кроссинговера**), который объединяет два гена и генерирует новое допустимое решение $\mathrm{crossover}: \Gamma^2\to\Gamma$.
 * Определить механизм **мутации** $\mathrm{mutate}: \Gamma\to\Gamma$.

Во многих случаях скрещивание и мутации — это довольно простые алгоритмы для манипулирования генами как числовыми последовательностями или битовыми векторами.

Конкретная реализация генетического алгоритма может варьироваться от случая к случаю, но общая структура выглядит следующим образом:

1. Выбрать начальную популяцию $G\subset\Gamma$
2. Случайным образом выбрать одну из операций, которые будут выполняться на этом шаге: скрещивание или мутация. 
  -  **Скрещивание**:
      * Случайным образом выбрать два гена $g_1, g_2 \in G$
      * Вычислить скрещивание $g=\mathrm{crossover}(g_1,g_2)$
      * Если $\mathrm{fit}(g)<\mathrm{fit}(g_1)$ или $\mathrm{fit}(g)<\mathrm{fit}(g_2)$ - заменить соответствующий ген в популяции на $g$.
  - **Мутация** - выбрать случайный ген $g\in G$ и заменить его на $\mathrm{mutate}(g)$
3. Повторять с шага 2, пока не получим достаточно маленькое значение $\mathrm{fit}$, или пока не будет достигнуто ограничение на количество шагов.

С помощью генетических алгоритмов обычно можно решать такие задачи:

1. Оптимизация расписания
2. Оптимальная упаковка
3. Оптимальная раскройка материалов
4. Ускорение полного перебора


## Задача 1: Разделение сокровищ по справедливости

**Задание**: 
Два человека нашли клад, в котором лежат бриллианты разного размера (и, соответственно, разной цены). Им нужно разделить клад на две части так, чтобы разница в цене была равна 0 (или минимальна).

**Формальное определение**: 
У нас есть набор чисел $S$. Нам нужно разделить его на два подмножества $S_1$ и $S_2$, такие, что $$\left|\sum_{i\in S_1}i - \sum_{j\in S_2}j\right|\to\min$$ и $S_1\cup S_2=S$, $S_1\cap S_2=\emptyset$.

Для начала, давайте определим множество $S$:

In [ ]:
N = 200
S = np.array([random.randint(1,10000) for _ in range(N)])
print(S)

(Если говорить строго, $S$ может не быть множеством, поскольку в нём могут повторяться элементы с одним и тем же значением. Однако, поскольку порядок элементов в кладе не очень существенен, о нем удобно думать как о множестве).

Закодируем каждое возможное решение задачи бинарным вектором $B\in\{0,1\}^N$, где номер на $i$-ой позиции показывает, к какому из множеств ($S_1$ или $S_2$) принадлежит $i$-й элемент изначального вектора $S$. Функция `generate` будет генерировать эти случайные двоичные векторы.

In [ ]:
def generate(S):
    return np.array([random.randint(0,1) for _ in S])

b = generate(S)
print(b)

Давайте теперь определим функцию `fit`, которая вычисляет «стоимость» решения. Это будет разница между суммой двух множеств, $S_1$ и $S_2$:

In [ ]:
def fit(B,S=S):
    c1 = (B*S).sum()
    c2 = ((1-B)*S).sum()
    return abs(c1-c2)

fit(b)

Теперь нам нужно определить функции для мутации и скрещивания:
* Для мутации мы выберем один случайный бит и инвертируем его (изменим с 0 на 1 и наоборот)
* Для скрещивания мы возьмем часть битов из одного вектора и часть битов из другого. Мы будем использовать ту же функцию `generate`, чтобы случайным образом выбирать, из какой входной маски брать биты.

In [ ]:
def mutate(b):
    x = b.copy()
    i = random.randint(0,len(b)-1)
    x[i] = 1-x[i]
    return x

def xover(b1,b2):
    x = generate(b1)
    return b1*x+b2*(1-x)

Создадим начальную популяцию решений $P$ размером `pop_size`:

In [ ]:
pop_size = 30
P = [generate(S) for _ in range(pop_size)]

Теперь опишем основную функция запуска эволюции. `n` — количество шагов эволюции. 

На каждом шаге:
* С вероятностью 30% выполняем мутацию и заменяем элемент с наихудшим значением `fit`- функции
* С вероятностью 70% выполняем скрещивание

Функция возвращает лучшее решение (ген, соответствующий лучшему решению) и историю минимальных значений фитнес-функции на каждой итерации - это позволяет отследить, насколько улучшаются решения в популяции.

In [ ]:
def evolve(P,S=S,n=2000):
    res = []
    for _ in range(n):
        f = min([fit(b) for b in P])
        res.append(f)
        if f==0:
            break
        if random.randint(1,10)<3:
            i = random.randint(0,len(P)-1)
            b = mutate(P[i])
            i = np.argmax([fit(z) for z in P])
            P[i] = b
        else:
            i = random.randint(0,len(P)-1)
            j = random.randint(0,len(P)-1)
            b = xover(P[i],P[j])
            if fit(b)<fit(P[i]):
                P[i]=b
            elif fit(b)<fit(P[j]):
                P[j]=b
            else:
                pass
    i = np.argmin([fit(b) for b in P])
    return (P[i],res)

(s,hist) = evolve(P)
print(s,fit(s))

Вы можете видеть, что нам удалось немного минимизировать функцию `fit`! Вот график, который показывает, как функция `fit` ведет себя во время процесса.

In [ ]:
plt.plot(hist)
plt.show()

## Задача 2: N ферзей

**Задание**:
Вам нужно расставить $N$ ферзей на шахматной доске размера $N\times N$ так, чтобы они не атаковали друг друга.

Прежде всего, решим задачу без генетических алгоритмов, используя полный перебор. Состояние доски можно представить в виде списка $L$, где $i$-е число в списке — это горизонтальное положение ферзя в $i$-м ряду. Совершенно очевидно, что в каждом решении будет только один ферзь в строке, и в каждой строке будет свой ферзь.

Нашей целью будет найти первое решение задачи, после чего мы прекратим поиск. Вы можете легко расширить эту функцию, чтобы генерировать все возможные позиции для ферзей.

Примерный ход решения такой:

1.	Ставим ферзя в первой строке на позицию 1
2.	Перебираем все позиции для ферзя в строке 2, пока он не будет поставлен так, что ферзь в строке 1 его не бьет
3.	Продолжаем делать то же самое с 3-ей строкой и т.д.
4.	Если мы обнаруживаем, что в какой-то момент не можем поставить ферзя – возвращаемся к предыдущей строке, и меняем положение ферзя на следующее из возможных.
5.	Таким образом в какой-то момент мы обнаружим, что дошли до конца доски и расставили все N ферзей так, что они не бьют друг друга.


In [14]:
N = 8

def checkbeats(i_new,j_new,l):
    for i,j in enumerate(l,start=1):
        if j==j_new:
            return False
        else:
            if abs(j-j_new) == i_new-i:
                return False
    return True

def nqueens(l,N=8,disp=True):
    if len(l)==N:
        if disp: print(l)
        return True
    else:
        for j in range(1,N+1):
            if checkbeats(len(l)+1,j,l):
                l.append(j)
                if nqueens(l,N,disp): return True
                else: l.pop()
        return False
            
nqueens([],30)


KeyboardInterrupt: 

Теперь давайте измерим, сколько времени потребуется, чтобы получить решение задачи с 20 ферзями:

In [15]:
%timeit nqueens([],20,False)

6.32 s ± 328 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Теперь решим ту же задачу с помощью генетического алгоритма. Это решение вдохновлено [этим сообщением в блоге](https://kushalvyas.github.io/gen_8Q.html).

Каждое решение будем представлять списком длины $N$, а в качестве фитнес-функции возьмем количество атакующих друг друга ферзей:

In [16]:
def fit(L):
    x=0
    for i1,j1 in enumerate(L,1):
        for i2,j2 in enumerate(L,1):
            if i2>i1:
                if j2==j1 or (abs(j2-j1)==i2-i1): x+=1
    return x

Поскольку вычисление функции приспособленности занимает много времени, давайте будем хранить каждое решение в популяции вместе со значением фитнес-функции. Сгенерируем начальную популяцию:

In [17]:
def generate_one(N):
    x = np.arange(1,N+1)
    np.random.shuffle(x)
    return (x,fit(x))

def generate(N,NP):
    return [generate_one(N) for _ in range(NP)]

generate(8,5)

[(array([2, 5, 4, 8, 1, 6, 3, 7]), 4),
 (array([7, 5, 1, 3, 4, 6, 8, 2]), 2),
 (array([3, 7, 5, 8, 6, 4, 1, 2]), 4),
 (array([6, 1, 5, 2, 8, 3, 4, 7]), 2),
 (array([4, 1, 5, 8, 7, 2, 6, 3]), 4)]

Теперь нам нужно определить функции мутации и кроссинговера (скрещивания). Кроссинговер объединил бы два гена вместе, разбив их в какой-то случайной точке и соединив вместе две части из разных генов.

In [23]:
def mutate(G):
    x=random.randint(0,len(G)-1)
    G[x]=random.randint(1,len(G))
    return G
    
def xover(G1,G2):
    x=random.randint(0,len(G1))
    return np.concatenate((G1[:x],G2[x:]))

xover([1,2,3,4],[5,6,7,8])

array([1, 2, 3, 8])

Мы улучшим процесс отбора генов, отбирая больше генов с лучшей функцией приспособленности:

In [25]:
def choose_rand(P):
    N=len(P[0][0])
    mf = N*(N-1)//2 # max fitness fn
    z = [mf-x[1] for x in P]
    tf = sum(z) # total fitness
    w = [x/tf for x in z]
    p = np.random.choice(len(P),2,False,p=w)
    return p[0],p[1]

def choose(P):
    def ch(w):
        p=[]
        while p==[]:
            r = random.random()
            p = [i for i,x in enumerate(P) if x[1]>=r]
        return random.choice(p)
    N=len(P[0][0])
    mf = N*(N-1)//2 # max fitness fn
    z = [mf-x[1] for x in P]
    tf = sum(z) # total fitness
    w = [x/tf for x in z]
    p1=p2=0
    while p1==p2:
        p1 = ch(w)
        p2 = ch(w)
    return p1,p2

Теперь давайте определим основной эволюционный цикл. Мы сделаем логику немного отличающейся от предыдущего примера - в генетических алгоритмах возможно применять творческий подход.

Цикл будет повторяться, пока мы не получим идеальное решение (функция приспособленности = 0). На каждом шаге мы будем брать текущую популяцию и создавать новое поколение того же размера. Это делается с помощью функции `nxgeneration`, используя следующие шаги:

1. Отбросить самые неподходящие решения - для этого есть функция `discard_unfit`
1. Добавить в генерацию несколько случайных решений (дополнительный аналог мутации)
1. Заполнить новое поколение размером `gen_size`, используя следующие шаги для каждого нового гена:
     - выбрать два случайных гена с вероятностью, пропорциональной фитнес-функции
     - рассчитать кроссинговер
     - применить мутацию с вероятностью `mutation_prob`

In [27]:
mutation_prob = 0.1

def discard_unfit(P):
    P.sort(key=lambda x:x[1])
    return P[:len(P)//3]

def nxgeneration(P):
    gen_size=len(P)
    P = discard_unfit(P)
    P.extend(generate(len(P[0][0]),3))
    new_gen = []
    for _ in range(gen_size):
        p1,p2 = choose_rand(P)
        n = xover(P[p1][0],P[p2][0])
        if random.random()<mutation_prob:
            n=mutate(n)
        nf = fit(n)
        new_gen.append((n,nf))
        '''
        if (nf<=P[p1][1]) or (nf<=P[p2][1]):
            new_gen.append((n,nf))
        elif (P[p1][1]<P[p2][1]):
            new_gen.append(P[p1])
        else:
            new_gen.append(P[p2])
        '''
    return new_gen
    
def genetic(N,pop_size=100):
    P = generate(N,pop_size)
    mf = min([x[1] for x in P])
    n=0
    while mf>0:
        #print("Generation {0}, fit={1}".format(n,mf))
        n+=1
        mf = min([x[1] for x in P])
        P = nxgeneration(P)
    mi = np.argmin([x[1] for x in P])
    return P[mi]

genetic(8)

(array([3, 6, 2, 7, 5, 1, 8, 4]), 0)

Интересно, что в большинстве случаев нам удается получить решение довольно быстро, но в некоторых редких случаях оптимизация достигает локального минимума, и процесс зависает надолго. Это важно учитывать при измерении среднего времени: хотя в большинстве случаев генетический алгоритм будет быстрее, чем полный поиск, в некоторых случаях его использование может занять больше времени. Чтобы преодолеть эту проблему, можно ограничить количество рассматриваемых поколений, и если решение не находится - перегенерировать популяцию.

In [28]:
%timeit genetic(10)

The slowest run took 146.08 times longer than the fastest. This could mean that an intermediate result is being cached.
10.4 s ± 12 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
